In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
categories, minNutrition, maxNutrition = gp.multidict({
    'protein':  [800, GRB.INFINITY],
    'fat':      [200, GRB.INFINITY],
    'Carbohydrate':   [700, GRB.INFINITY]})

In [3]:
foods, cost = gp.multidict({
    'FOOD_1': 45,
    'FOOD_2':   40,
    'FOOD_3':   85,
    'FOOD_4':   65 })
#     'macaroni':  2.09,
#     'pizza':     1.99,
#     'salad':     2.49,
#     'milk':      0.89,})

In [4]:
# Nutrition values for the foods
nutritionValues = {
    ('FOOD_1', 'protein'): 3,
    ('FOOD_1', 'fat'):  2,
    ('FOOD_1', 'Carbohydrate'): 6,
    
    ('FOOD_2',   'protein'): 4,
    ('FOOD_2',   'fat'):  2,
    ('FOOD_2',   'Carbohydrate'): 4,
    
    ('FOOD_3',   'protein'): 8,
    ('FOOD_3',   'fat'):  7,
    ('FOOD_3',   'Carbohydrate'): 7,
    
    ('FOOD_4',   'protein'): 6,
    ('FOOD_4',   'fat'):  5,
    ('FOOD_4',   'Carbohydrate'): 4,
}


In [5]:
# Model
m = gp.Model("diet")

Restricted license - for non-production use only - expires 2022-01-13


In [6]:
buy = m.addVars(foods, name="buy")

In [7]:
buy

{'FOOD_1': <gurobi.Var *Awaiting Model Update*>,
 'FOOD_2': <gurobi.Var *Awaiting Model Update*>,
 'FOOD_3': <gurobi.Var *Awaiting Model Update*>,
 'FOOD_4': <gurobi.Var *Awaiting Model Update*>}

In [8]:
# The objective is to minimize the costs
m.setObjective(buy.prod(cost), GRB.MINIMIZE)


In [9]:
# Nutrition constraints
m.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")

{'protein': <gurobi.Constr *Awaiting Model Update*>,
 'fat': <gurobi.Constr *Awaiting Model Update*>,
 'Carbohydrate': <gurobi.Constr *Awaiting Model Update*>}

In [10]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.objVal)
        print('\nBuy:')
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buy[f].x))
    else:
        print('No solution')


# Solve
m.optimize()
printSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x4b725178
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [4e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 8e+02]
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.125000e+02   0.000000e+00      0s
       1    8.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  8.000000000e+03

Cost: 8000

Buy:
FOOD_2 200
